# Large Movie Review Dataset
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. See the README file contained in the release for more details.


Large Movie Review Dataset v1.0
When using this dataset, please cite our ACL 2011 paper [bib].

Contact
For comments or questions on the dataset please contact Andrew Maas. As you publish papers using the dataset please notify us so we can post a link on this page.

Publications Using the Dataset
Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In [3]:
!pip install -q tensorflow-datasets

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to C:\Users\Jonathan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteUK5V5H\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\Jonathan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteUK5V5H\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\Jonathan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteUK5V5H\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\Jonathan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [20]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())
    
train_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [22]:
print(type(training_sentences))
print(len(training_sentences))
print(training_sentences[0])

<class 'list'>
25000
This is a big step down after the surprisingly enjoyable original. This sequel isn't nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.


In [23]:
print(type(training_labels))
print(len(training_labels))
print(training_labels[0])

<class 'list'>
25000
0


In [24]:
print(type(testing_sentences))
print(len(testing_sentences))
print(testing_sentences[0])

<class 'list'>
25000
It opens with your cliche overly long ship flying through space. All I could think at this point was "Spaceballs" and hoping there'd be a sticker on back that said "We break for Nobody." The movie then shows some cryogenic freezers with Vin Diesel's narration. I've always thought his voice sounded cool ever since I saw Fast and the Furious. From when I found out he was as criminal, I thought the movie was going to be cliche. It was. It was very cliche and fate seemed to be against them at every turn. Black out every 22 years. Lucky them, they land on that day. Aliens can only be in the darkness, hey it's a solar eclipse. As much as I thought it was too easy and just a cliche, the movie pulled through and kicked major @ss. I even went out and bought a copy of Pitch Black after seeing it. I really can't wait for Chronicles of Riddick.


In [25]:
print(type(testing_labels))
print(len(testing_labels))
print(testing_labels[0])

<class 'list'>
25000
1


In [14]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [16]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? for me this is a story that starts with some funny jokes regarding <OOV> <OOV> when he is travelling with a <OOV> and when he is sitting in business <OOV> the problem is that when you have been watching this movie for an hour you will see the same fantasies funny situations again and again and again it is to predictable it is more done as a tv story where you can go away and come back without missing anything br br i like felix <OOV> as frank but that is not enough even when it is a comedy it has to have more variations and some kind of message to it's audience br br
For me this is a story that starts with some funny jokes regarding Franks fanatasies when he is travelling with a staircase and when he is sitting in business meetings... The problem is that when you have been watching this movie for an hour you will see the same fantasies/funny situations again and again and again. It is to predictable. It is more done as a TV story where you can go away and come back without mi

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 102       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 10
model.fit(padded, train_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 251us/sample - loss: 0.5448 - accuracy: 0.7562 - val_loss: 0.3811 - val_accuracy: 0.8433
Epoch 2/10
25000/25000 [==============================] - 6s 242us/sample - loss: 0.3313 - accuracy: 0.8622 - val_loss: 0.3297 - val_accuracy: 0.8588
Epoch 3/10
25000/25000 [==============================] - 6s 239us/sample - loss: 0.2695 - accuracy: 0.8894 - val_loss: 0.3290 - val_accuracy: 0.8580
Epoch 4/10
25000/25000 [==============================] - 6s 228us/sample - loss: 0.2332 - accuracy: 0.9097 - val_loss: 0.3426 - val_accuracy: 0.8546
Epoch 5/10
25000/25000 [==============================] - 6s 229us/sample - loss: 0.2067 - accuracy: 0.9232 - val_loss: 0.3635 - val_accuracy: 0.8491
Epoch 6/10
25000/25000 [==============================] - 6s 236us/sample - loss: 0.1860 - accuracy: 0.9330 - val_loss: 0.3957 - val_accuracy: 0.8399
Epoch 7/10
25000/25000 [==========================

In [28]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [33]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [34]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

In [35]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
